In [2]:
import os
import pywt
from PIL import Image
import numpy as np

In [11]:
class FeatureVectorGenerator:
    def __init__(self, images_src_path, feature_vector_destination_path, iterations):
        self.images_src_path = images_src_path
        self.feature_vector_destination_path = feature_vector_destination_path
        self.iterations = iterations

    def get_vector_from_image(self, image):
        """
        get_vector_from_image obtiene el vector característico de la imagen image
        :param image: Imagen en formato vector.
        :param iterations: Entero que indica la cantidad de veces que se aplica el wavelet a la imagen.
        :return LL: Vector característico sin la compresión a 1D.
        :return LL.flatten(): Vector característico en 1D.
        """
        LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
        for _ in range(self.iterations - 1):
            LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
        return LL.flatten()

    def generate(self):
        for image_path in os.listdir(self.images_src_path):
            image = Image.open(self.images_src_path + image_path)
            fv = self.get_vector_from_image(image)
            fv = self.normalization(fv)
            folder = image_path[2]
            if folder == "0":
                folder = "10"
            label = ""
            for i in range(10):
                label += "1" if i == int(folder) - 1 else "0"
            f = open(self.feature_vector_destination_path + image_path[:-4] + ".txt", "w")
            f.write(label + "\n")
            for index, feature in enumerate(fv):
                f.write(str(feature))
                if index != len(fv) - 1:
                    f.write("\n")

    def normalization(self, fv):
        min_value = np.min(fv)
        max_value = np.max(fv)
        normalized_data = [(feature - min_value) / (max_value - min_value) for feature in fv]
        return normalized_data


fvg = FeatureVectorGenerator(images_src_path="resized_images/", feature_vector_destination_path="feature_vectors/", iterations=7)
fvg.generate()